In [24]:
import nltk
import pandas as pd
import numpy as np
nltk.download("popular")
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [25]:
df = pd.read_csv('./final1440_aapl.csv')

In [26]:
df

,Unnamed: 0,text,sentiment,label,stock_time
0,0,nape summit week will also feature the annual ...,0.050660,1.0,2017-12-08 05:00:00
1,1,from apple’s hugely anticipated iphone x to sa...,0.107128,-1.0,2017-12-11 05:00:00
2,2,"“they are not just the largest browser, but th...",0.034394,1.0,2017-12-12 05:00:00
3,3,the humanitarian crisis in the drc has placed ...,0.000000,-1.0,2017-12-13 05:00:00
4,4,cramer prefers finisar (nasdaq: fnsr ) after a...,0.032563,1.0,2017-12-15 05:00:00
...,...,...,...,...,...
281,281,"please try advanced search"", ""didyoumeantext"":...",0.017092,1.0,2019-01-28 05:00:00
282,282,"posted by darlene league on jan 28th, 2019 // ...",0.016114,1.0,2019-01-29 05:00:00
283,283,https://www.youtube.com/watch?v=6enwkf1sqog ta...,0.015341,1.0,2019-01-30 05:00:00
284,284,"r. edwardson on jan 30th, 2019 // no comments ...",0.016514,1.0,2019-01-31 05:00:00


In [27]:
# Load the text, sentiment, and labels from the dataframe
input_text = df['text']
input_sentiment = df['sentiment']
input_labels = df['label']

# Generate ngrams (2,2) with Count Vectorizer
count_vectorizer = CountVectorizer(max_df=0.8, min_df=0.02, stop_words=stop_words, ngram_range=(2,2))
vectorized_text = count_vectorizer.fit_transform(input_text)

# Use TfIdf to determine the frequency
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(vectorized_text).toarray()

# Use PCA to reduce dimensionality of the text dataset
pca = PCA(n_components=3)
pca_components = pca.fit_transform(X_tfidf)

# Combine text features with sentiment
final_input = np.hstack((pca_components, np.atleast_2d(input_sentiment).T))

# Create train and test split
X_train, X_test, y_train, y_test = train_test_split(final_input, input_labels, test_size=0.3, random_state=23)

# Scale the data
scaler = preprocessing.MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)


In [28]:
# Encode labels from [-1, 1] to [0, 1]
le = LabelEncoder()
y_train = le.fit_transform(y_train)

# Train model
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X = scaled_X_train,y =  y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [31]:
# Run test
y_pred = classifier.predict(scaled_X_test)
y_pred = le.inverse_transform(y_pred)

# Calculate training accuracy
train_accuracy = classifier.score(scaled_X_train, y_train)

# Calculate testing accuracy
test_accuracy = accuracy_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='binary')

# Get confusion matrix
cm = confusion_matrix(y_test, y_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)
print("F1 Score:", f1)
print("Confusion matrix:", cm)

Training Accuracy: 1.0
Testing Accuracy: 0.5813953488372093
F1 Score: 0.6896551724137931
Confusion matrix: [[10 18]
 [18 40]]
